In [2]:
from sentence_transformers import SentenceTransformer
# sentences = ["Cô giáo đang ăn kem", "giáo viên nữ của tôi đang ăn kem"]

model = SentenceTransformer('keepitreal/vietnamese-sbert')
# embeddings = model.encode(sentences)
# print(embeddings)


c:\Users\My_Pc\anaconda3\envs\Minh-AI\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
import time
import random
from googleapiclient.discovery import build
from tabulate import tabulate
from duckduckgo_search import DDGS
from sklearn.metrics.pairwise import cosine_similarity
import torch
import numpy as np
import unicodedata
import json
from bs4 import BeautifulSoup, NavigableString, Tag
import requests
import re
from pyvi import ViTokenizer, ViPosTagger
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urlunparse, urljoin

In [4]:
# Hàm để chuyển đổi văn bản thành embedding
def get_embedding(text, model):
    return model.encode(text)

In [5]:
def find_max_result_google_search(input_text, results, model):
    documents = []
    for temp in results:
        documents.append(temp['title'])
    if documents:
        # Chuyển đổi các văn bản thành embeddings
        embeddings = [get_embedding(doc, model) for doc in documents + [input_text]]
        cosine_similarities = cosine_similarity([embeddings[-1]], embeddings[:-1])
        # Chuyển độ tương đồng thành phần trăm
        similarities_percent = cosine_similarities[0] * 100
        # Tìm câu có độ tương đồng cao nhất
        max_similarity = max(similarities_percent)
        max_index = similarities_percent.tolist().index(max_similarity)
        # # In ra kết quả
        # for idx, sim in enumerate(similarities_percent):
        #     print(f"Độ tương đồng giữa văn bản đầu vào và văn bản {idx + 1}: {sim:.2f}%")
        # print(f"\nCâu có độ tương đồng cao nhất với văn bản đầu vào là câu {max_index + 1} với độ tương đồng {max_similarity:.2f}%: \"{documents[max_index]}\"")
        return results[max_index]
    return ''

In [6]:
def find_max_result_question(input_text, questions, model):
    if not questions:
        return input_text
    embeddings = [get_embedding(doc, model) for doc in questions + [input_text]]
    cosine_similarities = cosine_similarity([embeddings[-1]], embeddings[:-1])

    # Chuyển độ tương đồng thành phần trăm
    similarities_percent = cosine_similarities[0] * 100

    # Tìm câu có độ tương đồng cao nhất
    max_similarity = max(similarities_percent)
    max_index = similarities_percent.tolist().index(max_similarity)
    return questions[max_index]

In [7]:
def postagging_keyword(input_text):
    # Tách từ và gán nhãn từ loại
    words = ViTokenizer.tokenize(input_text)
    pos_tagged = ViPosTagger.postagging(words)
    
    # Lấy các từ loại danh từ (N), động từ (V), tính từ (A), danh từ riêng (Ny), số (M) và phân cách (F)
    keywords = []
    i = 0
    while i < len(pos_tagged[0]):
        word, pos = pos_tagged[0][i], pos_tagged[1][i]
        if pos in ['N', 'V', 'A', 'Ny']:
            keywords.append(word.lower())
        elif pos == 'M' or (pos == 'F' and word =='/'):
            combined_word = word
            j = i + 1
            while j < len(pos_tagged[0]) and pos_tagged[1][j] in ['M', 'F']:
                combined_word += pos_tagged[0][j]
                j += 1
            keywords.append(combined_word.lower())
            i = j - 1
        i += 1

    # Loại bỏ dấu gạch dưới
    keywords = [word.replace('_', ' ') for word in keywords]
    
    return keywords

In [8]:
def google_search(input_text):
    # black_list = '-site:https://thuvienphapluat.vn/cong-dong-dan-luat -site:https://thuvienphapluat.vn/van-ban'
    # black_list = '-site:https://lawnet.vn -site:https://vksndtc.gov.vn -site:thuenhanuoc.vn -site:https://lawnet.vn/vb -site:https://thuvienphapluat.vn/cong-dong-dan-luat -site:psyme.org -site:youtube.com -site:luatminhkhue.vn -site:https://thuvienphapluat.vn/van-ban -site:https://covanphaply.vn/bang -site:https://vietnamnet.vn'
    # + '+site:luatvietnam.vn ' 
    black_list = ' -site:https://luatvietnam.vn/luat-su-tu-van'
    search_query = 'site:luatvietnam.vn ' + input_text + black_list
    #  + ' ' + black_list
    # search_query = f'site:luatvietnam.vn {input_text} OR site:thuvienphapluat.vn {input_text} {black_list}'
    # 'site:luatvietnam.vn' + ' ' + 
    try:
        results = DDGS().text(
            keywords = search_query,
            region = 'vn-vi',
            safesearch= 'on',
            timelimit= '7d',
            max_results= 5
        )
        # Biểu thức regex để loại bỏ các URL không mong muốn
        regex_pattern = r'-d([1-9]|10)\.html'
        # Lọc kết quả bằng regex
        filtered_results = [result for result in results if not re.search(regex_pattern, result['href'])]
        return filtered_results
    except Exception as e:
        print(f"Rate limit or other exception: {e}")
        # Wait for a random time between 5 to 10 seconds
        time.sleep(random.uniform(5, 10))
        return google_search(input_text)

In [9]:
def crawl_data(data):  
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

    # Gửi yêu cầu HTTP GET đến URL
    response = requests.get(data.get('href'), headers=headers, verify= False)
    # , verify= False
    # response = requests.get(data.get('href'), headers=headers, verify=False, allow_redirects=True, max_redirects=10)
    soup_data = BeautifulSoup(response.content, 'html5lib', from_encoding= 'utf-8')
    return soup_data

In [10]:
# Hàm để tìm câu tương đồng nhất
def find_most_similar_sentence(postagging, results):
    original_words = set(postagging)
    # print(original_words)
    max_similarity = 0
    most_similar_sentence = ""
    
    for sentence in results:
        sentence_words = set(postagging_keyword(sentence['title'].lower()))
        common_words = original_words.intersection(sentence_words)
        similarity = len(common_words)
        
        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_sentence = sentence
    
    return most_similar_sentence

In [11]:
# Hàm để tìm câu tương đồng nhất
def find_most_similar_question(postagging, results):
    original_words = set(postagging)
    # print(original_words)
    max_similarity = 0
    most_similar_sentence = ""
    
    for sentence in results:
        sentence_words = set(postagging_keyword(sentence.lower()))
        common_words = original_words.intersection(sentence_words)
        similarity = len(common_words)
        
        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_sentence = sentence
    
    return most_similar_sentence

In [12]:
# xử lý nhiễu và lấy các phần câu hỏi
def find_h2_tags(soup_data):
    if soup_data is None:
        return []
    for toc in soup_data.find_all('div', class_="tinvanbanmoi"):
        toc.decompose()

    for toc in soup_data.find_all('figcaption'):
        toc.decompose()
        
    for toc in soup_data.find_all('div', class_="q_title"):
        toc.decompose()
    
    for toc in soup_data.find_all('p'):
        for toc1 in toc.find_all('strong'):
            if toc1.text == "Trả lời:":
                toc.decompose()
        
    for toc in soup_data.find_all('h3', style="text-align: center;"):
        toc.decompose()
    
    for toc in soup_data.find_all('p'):
        for temp in toc.find_all('span', class_="ptie2"):
            toc.decompose()
            
    for toc in soup_data.find_all('script'):
        toc.decompose()
    
    for toc in soup_data.find_all('div', class_="gm-gray-gradient p-3 mt-4 mb-4 related_posts"):
        toc.decompose()
        
    for toc in soup_data.find_all('p'):
        for toc1 in toc.find_all('em'):
            if toc1.text.startswith('Xem thêm:'):
                toc.decompose()
    
    for toc in soup_data.find_all('em'):
        if toc.text.startswith('Xem thêm:'):
            toc.decompose()
    for toc in soup_data.find_all('aside', class_="col-lg-3 col-md-3 col-sm-4 col-xs-12 sidebar"):
        toc.decompose()
        
    for toc in soup_data.find_all('div', class_="related-article"):
        toc.decompose()
        
    for toc in soup_data.find_all('div', class_="meta"):
        toc.decompose()
        
    for toc in soup_data.find_all('p'):
        for temp in toc.find_all('strong'):
            if temp.text.startswith('1900 6192'):
                toc.decompose()
    
    for toc in soup_data.find_all('div', style="text-align:center"):
        for toc1 in soup_data.find_all('em'):
            toc1.decompose()
                
    for toc in soup_data.find_all('div', style="background: #f9edd7; padding: 10px; margin-bottom: 10px;"):
            toc.decompose()
        
    for toc in soup_data.find_all('ul', class_="ulEvent"):
        toc.decompose()
        
    for toc in soup_data.find_all('div', class_="box_mucluc"):
        toc.decompose()
    
    for toc in soup_data.find_all('div', class_="tag-pos"):
        toc.decompose()
        
    for toc in soup_data.find_all('h2', class_="lead"):
        toc.decompose() 
    
    for toc in soup_data.find_all('section', class_="inner_section" ):
        toc.decompose()   
         
    for toc in soup_data.find_all('p', class_="Normal", style="text-align:right;"):
        toc.decompose()   
            
    # xử lý download
    for table_tag in soup_data.find_all('table', border="0", cellpadding="0", cellspacing="0"):
        if 'style' not in table_tag.attrs:
            # print(table_tag.text.lower())
            table_tag.decompose() 
        
    for toc in soup_data.find_all('p', style="text-align: center;"):
        toc.decompose()   
                   
    for toc in soup_data.find_all('div', style="text-align: center;"):
        toc.decompose()  
        
    for toc in soup_data.find_all('div', style="clear:both;text-align:center"):
        toc.decompose()  
  
    for toc in soup_data.find_all('div', style="text-align: right;"):
        toc.decompose() 
            
    for toc in soup_data.find_all('div', class_="panel panel-primary"):
        toc.decompose()   
    
    for toc in soup_data.find_all('p', style="text-align: right;"):
        toc.decompose()  
    
    for toc in soup_data.find_all('p', style="text-align: left;"):
        toc.decompose()  
    
    for toc in soup_data.find_all('p', style="text-align:right"):
        toc.decompose() 
          
    h2_tags = []
    for tags in soup_data.find_all('div', itemprop="articleBody"):
        h2_tags = tags.find_all('h2')
        
    if h2_tags == []:
        for tags in soup_data.find_all('div', class_="content_post_single"):
            h2_tags = tags.find_all('h2')
            
    if h2_tags == []:
        h2_tags = soup_data.find_all('h2')
        
    if h2_tags == []:
        for toc in soup_data.find_all('div', class_="d-flex justify-content-between time-control"):
            toc.decompose()
        
        for toc in soup_data.find_all('h6'):
            toc.decompose()
            
        for toc in soup_data.find_all('h5'):
            toc.decompose()
            
        for toc in soup_data.find_all('h4'):
            toc.decompose()
            
        for toc in soup_data.find_all('small'):
            toc.decompose()

        for toc in soup_data.find_all('a'):
            if "next_post" in toc.get('class', []):
                toc.decompose()

        for toc in soup_data.find_all('strong', class_="d-block mt-3 mb-3 sapo"):
            toc.decompose()

        for toc in soup_data.find_all('div', id="cau-hoi"):
            toc.decompose()

        for toc in soup_data.find_all('div', style="background-color: RGBA(196,231,249,0.3)"):
            toc.decompose()
        
        h2_tags = soup_data.find_all('h1')
    return h2_tags

In [13]:
# cắt từng cụm từ điều đến điều, khoản đến khoản, ..., xử lý các trường hợp khoản 1 đến khoản 5,...
def split_by_dieu_new1(text):
    if text is None:
        return[text]


    dieu_positions = [(match.start(), match.end()) for match in re.finditer(r'Điều\s+\d+[a-zA-Z0-9đĐ]*(?:,\s*\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)*(?:\s+và\s+\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)?', text, re.IGNORECASE)]
    if not dieu_positions:
        return [text]

    split_text = []
    current_position = 0

    for start, end in dieu_positions:
        if current_position < start:
            split_text.append(text[current_position:start].strip())
        split_text.append(text[start:end].strip())
        current_position = end

    if current_position < len(text):
        split_text.append(text[current_position:].strip())

    return split_text
################
def extract_dieu_err(text):
    # Define the pattern to match "Điều" followed by numbers and possibly letters or other characters
    pattern = r'Điều\s+\d+[a-zA-Z0-9đĐ]*(?:,\s*\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)*(?:\s+và\s+\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)?'

    # Find all matches of the pattern in the text
    matches = list(re.finditer(pattern, text, re.IGNORECASE))
    matched_phrases = [match.group(0) for match in matches]
    check_phrases = []

    # Loop through each phrase in matched_phrases
    for phrase in matched_phrases:
        if ',' in phrase or ' và ' in phrase:
            check_phrases.append(phrase)
    return check_phrases
##############
def process_dieu_parts(text):
    dieu_final = []
    for match in text:
        if ',' in match or ' và ' in match:
            splitted = re.split(r',| và ', match)
            for temp in splitted:
                if re.match(r'(?i)Điều\s+\d+', temp):
                    dieu_final.append(temp)
                else:
                    dieu_final.append('Điều ' + temp)

    return dieu_final
###############
def replace_dieu_in_text(text):
    if text is None:
        return[text]
    split_text = split_by_dieu_new1(text)
    matches = extract_dieu_err(text)
    if not matches:
        return text
    dieu_final = process_dieu_parts(matches)
    replaced_text = []
    for part in split_text:
        if re.match(r'(?i)Điều\s+\d+', part) and " và " in part:
            for dieu in dieu_final:
                replaced_text.append(dieu)
        else:
            replaced_text.append(part)

    return ' '.join(replaced_text)

#############
#############
########### khoản 1, 2,3  và .... + replace

def split_by_khoan_new1(text):
    if text is None:
        return[text]
    khoan_positions = [(match.start(), match.end()) for match in re.finditer(r'khoản\s+((?:[a-zA-Z0-9đĐ]+(?:\.\d+)?(?:,\s*| và |, và |))*[a-zA-Z0-9đĐ]+(?:\.\d+)?)(?=\s|,|\.|;|$)', text, re.IGNORECASE)]
    if not khoan_positions:
        return [text]

    split_text = []
    current_position = 0

    for start, end in khoan_positions:
        if current_position < start:
            split_text.append(text[current_position:start].strip())
        split_text.append(text[start:end].strip())
        current_position = end

    if current_position < len(text):
        split_text.append(text[current_position:].strip())

    return split_text

#####################
def extract_khoan_err(text):
    # Define the pattern to match "khoản" followed by numbers and possibly letters or other characters
    pattern = r'khoản\s+((?:[a-zA-Z0-9đĐ]+(?:\.\d+)?(?:,\s*| và |, và |))*[a-zA-Z0-9đĐ]+(?:\.\d+)?)(?=\s|,|\.|;|$)'

    # Find all matches of the pattern in the text
    matches = list(re.finditer(pattern, text, re.IGNORECASE))
    matched_phrases = [match.group(0) for match in matches]
    check_phrases = []

    # Loop through each phrase in matched_phrases
    for phrase in matched_phrases:
        if ',' in phrase or ' và ' in phrase:
            check_phrases.append(phrase)
    return check_phrases
################
def process_khoan_parts(text):
    khoan_final = []
    for match in text:
        if ',' in match or ' và ' in match:
            splitted = re.split(r',| và ', match)
            for temp in splitted:
                if re.match(r'(?i)khoản\s+((?:[a-zA-Z0-9đĐ]+(?:\.\d+)?)+)', temp):
                    khoan_final.append(temp)
                else:
                    khoan_final.append('khoản ' + temp)

    return khoan_final
###############

def replace_khoan_in_text(text):
    if text is None:
        return[text]
    split_text = split_by_khoan_new1(text)
    matches = extract_khoan_err(text)
    if not matches:
        return text
    khoan_final = process_khoan_parts(matches)
    replaced_text = []
    for part in split_text:
        if re.match(r'(?i)khoản\s+((?:[a-zA-Z0-9đĐ]+(?:\.\d+)?)+)', part) and " và " in part:
            for khoan in khoan_final:
                replaced_text.append(khoan)
        else:
            replaced_text.append(part)

    return ' '.join(replaced_text)
#############
#############
################# Điểm 1,2,3 và .... + replace

def split_by_diem_new1(text):
    def extract_diem_positions(text):
        return [(match.start(), match.end()) for match in re.finditer(r'điểm\s+((?:[a-zA-Z0-9đĐ]+(?:\.\d+)?(?:,\s+| và |, và |))*[a-zA-Z0-9đĐ]+(?:\.\d+)?)(?=\s|,|\.|;|$|\s+và\s+[a-zA-ZđĐ0-9])', text, re.IGNORECASE)]

    diem_positions = extract_diem_positions(text)
    if not diem_positions:
        return [text]

    split_text = []
    current_position = 0

    for start, end in diem_positions:
        if current_position < start:
            split_text.append(text[current_position:start].strip())
        split_text.append(text[start:end].strip())
        current_position = end

    if current_position < len(text):
        split_text.append(text[current_position:].strip())

    return split_text
###########################
def extract_diem_err(text):
    # Define the pattern to match "điểm" followed by numbers and possibly letters or other characters
    pattern = r'điểm\s+((?:[a-zA-Z0-9đĐ]+(?:\.\d+)?(?:,\s+| và |, và |))*[a-zA-Z0-9đĐ]+(?:\.\d+)?)(?=\s|,|\.|;|$|\s+và\s+[a-zA-ZđĐ0-9])'

    # Find all matches of the pattern in the text
    matches = list(re.finditer(pattern, text, re.IGNORECASE))
    matched_phrases = [match.group(0) for match in matches]
    check_phrases = []

    # Loop through each phrase in matched_phrases
    for phrase in matched_phrases:
        if ',' in phrase or ' và ' in phrase:
            check_phrases.append(phrase)
    return check_phrases
#########################
def process_diem_parts(text):
    diem_final = []
    for match in text:
        if ',' in match or ' và ' in match:
            splitted = re.split(r',| và ', match)
            for temp in splitted:
                if re.match(r'(?i)điểm\s+((?:[a-zA-Z0-9đĐ]+(?:\.\d+)?)+)', temp):
                    diem_final.append(temp)
                else:
                    diem_final.append('điểm ' + temp)

    return diem_final
#########################
def replace_diem_in_text(text):
    if text is None:
        return[text]
    split_text = split_by_diem_new1(text)
    matches = extract_diem_err(text)
    if not matches:
        return text
    diem_final = process_diem_parts(matches)
    replaced_text = []
    for part in split_text:
        if re.match(r'(?i)điểm\s+((?:[a-zA-Z0-9đĐ]+(?:\.\d+)?)+)', part) and " và " in part:
            for diem in diem_final:
                replaced_text.append(diem)
        else:
            replaced_text.append(part)

    return ' '.join(replaced_text)

def split_and_transform_dieu_ranges(text):
    if text is None:
        return [text]

    dieu_positions = [(match.start(), match.end()) for match in re.finditer(r'Điều\s+\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*', text, re.IGNORECASE)]

    if not dieu_positions:
        return [text]

    new_split_text = []
    current_position = 0
    start_dieu = None

    for i, (start, end) in enumerate(dieu_positions):
        if current_position < start:
            split_text_part = text[current_position:end].strip()
            if "từ Điều" in split_text_part:
                start_dieu_match = re.search(r'Điều\s+(\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)', split_text_part, re.IGNORECASE)
                if start_dieu_match:
                    start_dieu = start_dieu_match.group(1)
            elif "đến Điều" in split_text_part or "tới Điều" in split_text_part:
                end_dieu_match = re.search(r'Điều\s+(\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)', split_text_part, re.IGNORECASE)
                if end_dieu_match:
                    end_dieu = end_dieu_match.group(1)
                    start_index = int(re.sub("[a-zA-ZđĐ]", "", start_dieu)) if start_dieu else None
                    end_index = int(re.sub("[a-zA-ZđĐ]", "", end_dieu)) if end_dieu else None
                    if start_index is not None and end_index is not None:
                        for d in range(start_index, end_index + 1):
                            new_split_text.append(f"Điều {d}")
            else:
                new_split_text.append(split_text_part)

        current_position = end

    if current_position < len(text):
        split_text_part = text[current_position:].strip()
        if "từ Điều" in split_text_part:
            start_dieu_match = re.search(r'Điều\s+(\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)', split_text_part, re.IGNORECASE)
            if start_dieu_match:
                start_dieu = start_dieu_match.group(1)
        elif "đến Điều" in split_text_part or "tới Điều" in split_text_part:
            end_dieu_match = re.search(r'Điều\s+(\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)', split_text_part, re.IGNORECASE)
            if end_dieu_match:
                end_dieu = end_dieu_match.group(1)
                start_index = int(re.sub("[a-zA-ZđĐ]", "", start_dieu)) if start_dieu else None
                end_index = int(re.sub("[a-zA-ZđĐ]", "", end_dieu)) if end_dieu else None
                if start_index is not None and end_index is not None:
                    for d in range(start_index, end_index + 1):
                        new_split_text.append(f"Điều {d}")
        else:
            new_split_text.append(split_text_part)

    return new_split_text

def replace_dieu_ranges(text):
    # Tìm khoảng "từ Điều X đến Điều Y" hoặc "từ Điều X tới Điều Y"
    if text is None:
        return[text]
    dieu_range_match = re.search(r'từ Điều \d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)* (?:đến|tới) Điều \d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*', text, re.IGNORECASE)
    if not dieu_range_match:
        return text

    dieu_range_text = dieu_range_match.group(0)
    dieu_list = split_and_transform_dieu_ranges(dieu_range_text)

    # Tạo chuỗi mới từ danh sách các Điều
    dieu_list_str = ', '.join(dieu_list)

    # Thay thế đoạn văn bản "từ Điều X đến Điều Y" hoặc "từ Điều X tới Điều Y" bằng danh sách các Điều
    new_text = text.replace(dieu_range_text, dieu_list_str)

    return new_text

def split_and_transform_khoan_ranges(text):
    if text is None:
        return [text]


    khoan_positions = [(match.start(), match.end()) for match in re.finditer(r'khoản\s+\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*', text, re.IGNORECASE)]
    if not khoan_positions:
        return [text]

    new_split_text = []
    current_position = 0
    start_khoan = None

    for i, (start, end) in enumerate(khoan_positions):
        if current_position < start:
            split_text_part = text[current_position:end].strip()
            if "từ khoản" in split_text_part.lower():
                start_khoan = re.search(r'khoản\s+(\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)', split_text_part, re.IGNORECASE).group(1)
            elif "đến khoản" in split_text_part.lower() or "tới khoản" in split_text_part.lower():
                end_khoan = re.search(r'khoản\s+(\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)', split_text_part, re.IGNORECASE).group(1)
                start_index = int(re.sub("[a-zA-ZđĐ]", "", start_khoan))
                end_index = int(re.sub("[a-zA-ZđĐ]", "", end_khoan))
                for d in range(start_index, end_index + 1):
                    new_split_text.append(f"khoản {d}")
            else:
                new_split_text.append(split_text_part)
        current_position = end

    if current_position < len(text):
        split_text_part = text[current_position:].strip()
        if "từ khoản" in split_text_part.lower():
            start_khoan = re.search(r'khoản\s+(\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)', split_text_part, re.IGNORECASE).group(1)
        elif "đến khoản" in split_text_part.lower() or "tới khoản" in split_text_part.lower():
            end_khoan = re.search(r'khoản\s+(\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)', split_text_part, re.IGNORECASE).group(1)
            start_index = int(re.sub("[a-zA-ZđĐ]", "", start_khoan))
            end_index = int(re.sub("[a-zA-ZđĐ]", "", end_khoan))
            for d in range(start_index, end_index + 1):
                new_split_text.append(f"khoản {d}")
        else:
            new_split_text.append(split_text_part)

    return new_split_text

def replace_khoan_ranges(text):
    # Tìm khoảng "từ khoản X đến khoản Y" hoặc "từ khoản X tới khoản Y"
    if text is None:
        return[text]
    khoan_range_matches = re.finditer(r'từ khoản \d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)* (?:đến|tới) khoản \d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*', text, re.IGNORECASE)

    new_text = text
    offset = 0

    for match in khoan_range_matches:
        khoan_range_text = match.group(0)
        khoan_list = split_and_transform_khoan_ranges(khoan_range_text)

        # Tạo chuỗi mới từ danh sách các khoản
        khoan_list_str = ', '.join(khoan_list)

        # Tính toán vị trí mới sau khi thay thế
        start, end = match.start() + offset, match.end() + offset
        new_text = new_text[:start] + khoan_list_str + new_text[end:]

        # Cập nhật offset
        offset += len(khoan_list_str) - len(khoan_range_text)

    return new_text

def split_and_transform_diem_ranges(text):
    if text is None:
        return [text]


    diem_positions = [(match.start(), match.end()) for match in re.finditer(r'điểm\s+\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*', text, re.IGNORECASE)]
    if not diem_positions:
        return [text]

    new_split_text = []
    current_position = 0
    start_diem = None

    for i, (start, end) in enumerate(diem_positions):
        if current_position < start:
            split_text_part = text[current_position:end].strip()
            if "từ điểm" in split_text_part.lower():
                start_diem = re.search(r'điểm\s+(\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)', split_text_part, re.IGNORECASE).group(1)
            elif "đến điểm" in split_text_part.lower() or "tới điểm" in split_text_part.lower():
                end_diem = re.search(r'điểm\s+(\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)', split_text_part, re.IGNORECASE).group(1)
                start_index = int(re.sub("[a-zA-ZđĐ]", "", start_diem))
                end_index = int(re.sub("[a-zA-ZđĐ]", "", end_diem))
                for d in range(start_index, end_index + 1):
                    new_split_text.append(f"điểm {d}")
            else:
                new_split_text.append(split_text_part)
        current_position = end

    if current_position < len(text):
        split_text_part = text[current_position:].strip()
        if "từ điểm" in split_text_part.lower():
            start_diem = re.search(r'điểm\s+(\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)', split_text_part, re.IGNORECASE).group(1)
        elif "đến điểm" in split_text_part.lower() or "tới điểm" in split_text_part.lower():
            end_diem = re.search(r'điểm\s+(\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)', split_text_part, re.IGNORECASE).group(1)
            start_index = int(re.sub("[a-zA-ZđĐ]", "", start_diem))
            end_index = int(re.sub("[a-zA-ZđĐ]", "", end_diem))
            for d in range(start_index, end_index + 1):
                new_split_text.append(f"điểm {d}")
        else:
            new_split_text.append(split_text_part)

    return new_split_text

def replace_diem_ranges(text):
    # Tìm khoảng "từ điểm X đến điểm Y" hoặc "từ điểm X tới điểm Y"
    if text is None:
        return[text]
    diem_range_matches = re.finditer(r'từ điểm \d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)* (?:đến|tới) điểm \d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*', text, re.IGNORECASE)

    new_text = text
    offset = 0

    for match in diem_range_matches:
        diem_range_text = match.group(0)
        diem_list = split_and_transform_diem_ranges(diem_range_text)

        # Tạo chuỗi mới từ danh sách các điểm
        diem_list_str = ', '.join(diem_list)

        # Tính toán vị trí mới sau khi thay thế
        start, end = match.start() + offset, match.end() + offset
        new_text = new_text[:start] + diem_list_str + new_text[end:]

        # Cập nhật offset
        offset += len(diem_list_str) - len(diem_range_text)

    return new_text

########### Cắt từ điều này tới điều kia


def split_by_dieu(text):
    if text is None:
        return[text]

    text = ' ' + text
    dieu_positions = [(match.start(), match.end()) for match in re.finditer(r'Điều\s+\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*', text, re.IGNORECASE)]
    if not dieu_positions:
        return [text]

    split_text = []
    current_position = 0

    for i, (start, end) in enumerate(dieu_positions):
        if current_position < start:
            split_text.append(text[current_position:end].strip())
        current_position = end

    if current_position < len(text):
        split_text.append(text[current_position:].strip())

    return split_text

########### Cắt từ khoản này tới khoản kia

def split_by_khoan(text):
    if text is None:
            return[text]

    text = ' ' + text
    khoan_positions = [(match.start(), match.end()) for match in re.finditer(r'khoản\s+((?:[a-zA-Z0-9đĐ]+(?:\.\d+)?(?:,\s*| và |, và |))*[a-zA-Z0-9đĐ]+(?:\.\d+)?)(?=\s|,|\.|;|$)', text, re.IGNORECASE)]
    if not khoan_positions:
        return [text]

    split_text = []
    current_position = 0

    first_khoan_start = khoan_positions[0][0]
    if first_khoan_start > 0:
        split_text.append(text[:first_khoan_start].strip())

    for i, (start, end) in enumerate(khoan_positions):
        if current_position < start:
            split_text.append(text[current_position:end].strip())
        current_position = end

    if current_position < len(text):
        split_text.append(text[current_position:].strip())

    return split_text

def processing_ccpl_full(text):
    if text is None:
        return [text]
    check = replace_dieu_ranges(replace_khoan_ranges(replace_diem_ranges(text)))
    check2 = replace_dieu_in_text(replace_khoan_in_text(replace_diem_in_text(check)))

    return check2

# Hàm trích xuất điều
def extract_dieu(text):
    # Find individual "Điều" references
    dieu_list = re.findall(r'Điều\s+(\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)', text, re.IGNORECASE)

    # Find joined "Điều" lists
    joined_dieu_list = re.findall(r'Điều\s+\d+[a-zA-Z0-9đĐ]*(?:,\s*\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)*(?:\s+và\s+\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*)?', text, re.IGNORECASE)

    # Process joined "Điều" lists
    for item in joined_dieu_list:
        # Find all "Điều" references in the joined list
        joined_dieu_refs = re.findall(r'\d+[a-zA-Z0-9đĐ]*(?:\.\d+[a-zA-Z0-9đĐ]*)*', item)
        # Add only the unique "Điều" references to the main list
        for ref in joined_dieu_refs:
            if ref not in dieu_list:
                dieu_list.append(ref)
    return dieu_list

# Hàm trích xuất khoản
def extract_khoan(text):
    pattern = r'khoản\s+((?:[a-zA-Z0-9đĐ]+(?:\.\d+)?(?:,\s*| và |, và |))*[a-zA-Z0-9đĐ]+(?:\.\d+)?)(?=\s|,|\.|;|$)'
    matches = re.findall(pattern, text, re.IGNORECASE)
    final_khoan_list = []
    for match in matches:
        parts = re.split(r',\s*|\s+và\s+', match)
        for part in parts:
            part = part.strip().strip('.').lower()
            if '-' in part:
                start, end = part.split('-')
                for idx in range(int(start), int(end) + 1):
                    final_khoan_list.append(str(idx))
            elif len(part) == 1 and part.isalpha() or part[0].isdigit():
                final_khoan_list.append(part)
    return final_khoan_list

# Hàm trích xuất điểm
def extract_diem(text):
    pattern = r'điểm\s+((?:[a-zA-Z0-9đĐ]+(?:\.\d+)?(?:,\s+| và |, và |))*[a-zA-Z0-9đĐ]+(?:\.\d+)?)(?=\s|,|\.|;|$|\s+và\s+[a-zA-ZđĐ0-9])'
    matches = re.findall(pattern, text, re.IGNORECASE)
    final_diem_list = []
    for match in matches:
        parts = re.split(r',\s*|\s+và\s+', match)
        for part in parts:
            part = part.strip().strip('.').lower()
            if re.match(r'^[a-zA-ZđĐ]\d*(\.\d+)*$', part) or re.match(r'^\d+(\.\d+)*$', part):
                final_diem_list.append(part)
    return final_diem_list

# hàm xử lý đầu ra ccpl
def processing_output_ccpl(input_data):
    output_array = []
    for dieu in input_data:
        dieu_value = ", ".join(dieu["Dieu"])
        if dieu["Khoan"]:
            for khoan in dieu["Khoan"]:
                for khoan_item in khoan["Khoan"]:
                    if khoan["Diem"]:
                        for diem in khoan["Diem"]:
                            for diem_item in diem:
                                output_array.extend([
                                    dieu_value,
                                    khoan_item,
                                    diem_item
                                ])
                    else:
                        output_array.extend([
                            dieu_value,
                            khoan_item,
                            "0"
                        ])
        else:
            output_array.extend([
                dieu_value,
                "0",
                "0"
            ])
    if output_array:
        return output_array

    else:
        return []

In [14]:
# Function to remove duplicate ccpl entries
def remove_duplicates_ccpls(data):
    for item in data:
        seen = set()
        unique_ccpls = []
        for ccpl in item['ccpls']:
            if ccpl['Dieu']:  # Kiểm tra nếu 'Dieu' không phải là rỗng
                ccpl_tuple = (ccpl['LawTitle'], ccpl['Dieu'], ccpl['Khoan'], ccpl['Diem'])
                # ccpl['LawID'], 
                if ccpl_tuple not in seen:
                    seen.add(ccpl_tuple)
                    unique_ccpls.append(ccpl)
        item['ccpls'] = unique_ccpls
    return data

In [15]:
# format các dạng bảng
def table_to_markdown(table):
    if table is None:
        return ""
    rows = []
    headers = []

    for row in table.find_all('tr'):
        row_data = []
        for cell in row.find_all(['th', 'td']):
            cell_text = cell.get_text(strip=True)
            row_data.append(cell_text)
        if not headers:
            headers = row_data
        else:
            rows.append(row_data)

    return tabulate(rows, headers, tablefmt='pipe')

In [16]:
# hàm lấy q,a,ccpl
def processing_1(h2_tags, soup_data): #response.content
    # Tìm tất cả các thẻ <h2>
    questions_and_answers1 = []
    if len(h2_tags) == 1:
        answer_text = ""
        for i in range(len(h2_tags)):
            h2_current = h2_tags[i]
            temp_results = []
            next_tag = h2_tags[i].find_next_sibling()
            gay_flag = True
            while next_tag and next_tag.name != h2_current.name:
                if next_tag.name == 'p':
                    p_tag = next_tag.text
                    if (re.search(r"thư\s+viện\s+pháp\s+luật", next_tag.text.lower())):
                        gay_flag = False
                    if next_tag.find('a') and not next_tag.text.startswith('>>') and gay_flag == True:
                        for a_tag in next_tag.find_all('a'):
                            href = a_tag.get('href')
                            if href and (re.search(r'-d([1-9]|10)\.html', href) or '.aspx' in href):
                                final = processing_ccpl_full(a_tag.text)
                                positions = split_by_dieu(final)
                                # test = p_tag.split(positions)
                                # print(positions)
                                lawtitle = positions[-1]
                                test = p_tag.split(lawtitle)
                                # print(lawtitle)
                                anchor_match = re.search(r'anchor=(\w+)', href)
                                anchor = anchor_match.group(1) if anchor_match else "0"
                                if "điều" in test[0].lower():
                                    p_tag = p_tag.replace(test[0], '', 1)
                                    final = processing_ccpl_full(test[0])
                                    positions = split_by_dieu(final)
                                    data_ccpl = []
                                    for dieu in positions:
                                        if "điều" in dieu.lower():
                                            dieu_dict = {"Dieu": extract_dieu(dieu), "Khoan": []}
                                            # Tìm anchor trong href
                                            khoan_list = split_by_khoan(dieu)
                                            for khoan in khoan_list:
                                                if "khoản" in khoan.lower():
                                                    khoan_dict = {"Khoan": extract_khoan(khoan), "Diem": []}
                                                    if "điểm" in khoan.lower():
                                                        diem_dict = extract_diem(khoan)
                                                        khoan_dict["Diem"].append(diem_dict)
                                                    dieu_dict["Khoan"].append(khoan_dict)
                                            data_ccpl.append(dieu_dict)
                                            # print(data_ccpl)
                                            # Tạo một từ điển mới và gán giá trị từ mảng
                                            ccpl_temp = processing_output_ccpl(data_ccpl)
                                            if len(ccpl_temp) > 3:
                                                for i in range(0, len(ccpl_temp), 3):
                                                    # Lấy 3 phần tử từ chỉ mục i đến i+3 (không bị lỗi nếu kích thước không đủ)
                                                    chunk = ccpl_temp[i:i+3]
                                                    anchor_match = re.search(r'anchor=(\w+)', href)
                                                    law_id_match = re.search(r'-(\d+)\.aspx', href)
                                                    anchor = anchor_match.group(1) if anchor_match else "0"
                                                    law_id = law_id_match.group(1) if law_id_match else "0"
                                                    # print(ccpl_temp)
                                                    if len(chunk) > 0:
                                                        dictionary = {
                                                            # "url": href,
                                                            # "anchor": anchor,
                                                            # "LawID": law_id,
                                                            "LawTitle": lawtitle,
                                                            "Dieu": chunk[0],
                                                            "Khoan": chunk[1],
                                                            "Diem": chunk[2]
                                                        }
                                                        temp_results.append(dictionary)
                                            else:
                                                anchor_match = re.search(r'anchor=(\w+)', href)
                                                law_id_match = re.search(r'-(\d+)\.aspx', href)
                                                anchor = anchor_match.group(1) if anchor_match else "0"
                                                law_id = law_id_match.group(1) if law_id_match else "0"
                                                if len(ccpl_temp) > 0:
                                                    dictionary = {
                                                        # "url": href,
                                                        # "anchor": anchor,
                                                        # "LawID": law_id,
                                                        "LawTitle": lawtitle,
                                                        "Dieu": ccpl_temp[0] or "0",
                                                        "Khoan": ccpl_temp[1],
                                                        "Diem": ccpl_temp[2]
                                                    }
                                                    temp_results.append(dictionary)

                                else:
                                    temp_else = {}
                                    anchor_match = re.search(r'anchor=(\w+)', href)
                                    law_id_match = re.search(r'-(\d+)\.aspx', href)
                                    anchor = anchor_match.group(1) if anchor_match else "0"
                                    law_id = law_id_match.group(1) if law_id_match else "0"
                                    # law_title = a_tag.get_text()
                                    # temp_else["url"] = href
                                    # temp_else["anchor"] = anchor
                                    # temp_else["LawID"] = law_id
                                    temp_else["LawTitle"] = lawtitle
                                    temp_else["Dieu"] = "0"
                                    temp_else["Khoan"] = "0"
                                    temp_else["Diem"] = "0"
                                    temp_results.append(temp_else)
                        gay_flag = True
                next_tag = next_tag.find_next_sibling()
            ccpls = []
            ccpls.extend(temp_results)
        answer = soup_data.find('div', id="tra-loi")

        if not answer:
            answer = soup_data.find('section', class_ = "news-content")
            
        if not answer:
            answer = soup_data.find('div', id="content_article")
        if not answer:
            answer = soup_data.find('div', id="article-content")
            
        if not answer:
            answer = soup_data.find('div', id="news-bodyhtml")
        
        if not answer:
            answer = soup_data.find('div', class_="article-detail")
        
        if not answer:
            answer = soup_data.find('div', class_="content-detail")  
            
        if not answer:
            answer = soup_data.find('div', class_="entry-content single-page")  
            
        if not answer:
            answer = soup_data.find('div', class_="noidung-chitiet")  
        if not answer:
            answer = soup_data.find('div', class_="the-article-body entry")  
        if not answer:
            answer = soup_data.find('div', class_="entry-hoi-dap")  

        # # wikihow
        # if not answer:
        #     answer = soup_data.find('div', id="mf-section-1")       
            
        #trang báo
        if not answer:
            answer = soup_data.find('article', class_="fck_detail")
  
        if answer:    
            # print(answer)
            for temp in answer:
                if not isinstance(temp, Tag) and temp.name == 'table':
                    answer_text += table_to_markdown(temp) + "\n"
                    # print(answer_text)
                else:
                    answer_text += temp.text + "\n"
                
            questions_and_answers1.append({
                "question": unicodedata.normalize("NFC", h2_current.text),
                "answer": re.sub(r'\n{2,}', '\n', answer_text.strip()),
                "ccpls": []
            })
    else:
        for i in range(len(h2_tags)):
            h2_current = h2_tags[i]
            # Lấy văn bản cho câu trả lời
            answer_text = ""
            next_element = h2_current.next_sibling

            while next_element and (not isinstance(next_element, Tag) or next_element.name != h2_current.name):
                if next_element and next_element.name == 'table':
                    answer_text += table_to_markdown(next_element) + "\n"
                    # answer_text += answer.strip() + "\n"
                    # Kiểm tra nếu next_element là NavigableString
                    # print(answer_text)
                elif isinstance(next_element, NavigableString):
                    answer = unicodedata.normalize("NFC", next_element)
                    answer_text += answer.strip() + "\n"
                elif next_element and next_element.name:
                    answer = unicodedata.normalize("NFC", next_element.get_text())
                    answer_text += answer.strip() + "\n"
                next_element = next_element.next_sibling

            temp_results = []
            next_tag = h2_tags[i].find_next_sibling()
            gay_flag = True
            while next_tag and next_tag.name != h2_current.name:
                if next_tag.name == 'p':
                    p_tag = next_tag.text
                    if (re.search(r"thư\s+viện\s+pháp\s+luật", next_tag.text.lower())):
                        gay_flag = False
                    if next_tag.find('a') and not next_tag.text.startswith('>>') and gay_flag == True:
                        for a_tag in next_tag.find_all('a'):
                            href = a_tag.get('href')
                            if href and (re.search(r'-d([1-9]|10)\.html', href) or '.aspx' in href):
                                final = processing_ccpl_full(a_tag.text)
                                positions = split_by_dieu(final)
                                lawtitle = positions[-1]
                                test = p_tag.split(lawtitle)
                                # print(lawtitle)
                                anchor_match = re.search(r'anchor=(\w+)', href)
                                anchor = anchor_match.group(1) if anchor_match else "0"
                                if "điều" in test[0].lower():
                                    p_tag = p_tag.replace(test[0], '', 1)
                                    final = processing_ccpl_full(test[0])
                                    positions = split_by_dieu(final)
                                    data_ccpl = []
                                    for dieu in positions:
                                        if "điều" in dieu.lower():
                                            dieu_dict = {"Dieu": extract_dieu(dieu), "Khoan": []}
                                            # Tìm anchor trong href
                                            khoan_list = split_by_khoan(dieu)
                                            for khoan in khoan_list:
                                                if "khoản" in khoan.lower():
                                                    khoan_dict = {"Khoan": extract_khoan(khoan), "Diem": []}
                                                    if "điểm" in khoan.lower():
                                                        diem_dict = extract_diem(khoan)
                                                        khoan_dict["Diem"].append(diem_dict)
                                                    dieu_dict["Khoan"].append(khoan_dict)
                                            data_ccpl.append(dieu_dict)
                                            # print(data_ccpl)
                                            # Tạo một từ điển mới và gán giá trị từ mảng
                                            ccpl_temp = processing_output_ccpl(data_ccpl)
                                            if len(ccpl_temp) > 3:
                                                for i in range(0, len(ccpl_temp), 3):
                                                    # Lấy 3 phần tử từ chỉ mục i đến i+3 (không bị lỗi nếu kích thước không đủ)
                                                    chunk = ccpl_temp[i:i+3]
                                                    anchor_match = re.search(r'anchor=(\w+)', href)
                                                    law_id_match = re.search(r'-(\d+)\.aspx', href)
                                                    anchor = anchor_match.group(1) if anchor_match else "0"
                                                    law_id = law_id_match.group(1) if law_id_match else "0"
                                                    # print(ccpl_temp)
                                                    if len(chunk) > 0:
                                                        dictionary = {
                                                            # "url": href,
                                                            # "anchor": anchor,
                                                            # "LawID": law_id,
                                                            "LawTitle": lawtitle,
                                                            "Dieu": chunk[0],
                                                            "Khoan": chunk[1],
                                                            "Diem": chunk[2]
                                                        }
                                                        temp_results.append(dictionary)
                                            else:
                                                anchor_match = re.search(r'anchor=(\w+)', href)
                                                law_id_match = re.search(r'-(\d+)\.aspx', href)
                                                anchor = anchor_match.group(1) if anchor_match else "0"
                                                law_id = law_id_match.group(1) if law_id_match else "0"
                                                if len(ccpl_temp) > 0:
                                                    dictionary = {
                                                        # "url": href,
                                                        # "anchor": anchor,
                                                        # "LawID": law_id,
                                                        "LawTitle": lawtitle,
                                                        "Dieu": ccpl_temp[0] or "0",
                                                        "Khoan": ccpl_temp[1],
                                                        "Diem": ccpl_temp[2]
                                                    }
                                                    temp_results.append(dictionary)

                                else:
                                    temp_else = {}
                                    anchor_match = re.search(r'anchor=(\w+)', href)
                                    law_id_match = re.search(r'-(\d+)\.aspx', href)
                                    anchor = anchor_match.group(1) if anchor_match else "0"
                                    law_id = law_id_match.group(1) if law_id_match else "0"
                                    # law_title = a_tag.get_text()
                                    # temp_else["url"] = href
                                    # temp_else["anchor"] = anchor
                                    # temp_else["LawID"] = law_id
                                    temp_else["LawTitle"] = lawtitle
                                    temp_else["Dieu"] = "0"
                                    temp_else["Khoan"] = "0"
                                    temp_else["Diem"] = "0"
                                    temp_results.append(temp_else)
                        gay_flag = True
                next_tag = next_tag.find_next_sibling()
            ccpls = []
            ccpls.extend(temp_results)
            questions_and_answers1.append({
                "question": unicodedata.normalize("NFC", h2_current.text),
                "answer": re.sub(r'\n{2,}', '\n', answer_text.strip()),
                "ccpls": ccpls
            })
    # Danh sách các mẫu regex cần tìm và xóa từ đó đến hết câu trả lời
    patterns = [
        re.compile(r"\s*\n{2}\s*Về\s*vấn\s*đề\s*này,"),  # Khớp với ít nhất 2 ký tự xuống dòng trước và giữa các từ
        re.compile(r"Về vấn đề này,\s*THƯ VIỆN PHÁP LUẬT\s*giải đáp như sau:"),
        re.compile(r"\*\*\*\*", re.MULTILINE),
        re.compile(r"THƯ VIỆN PHÁP LUẬT"),
        re.compile(r"^Xem thêm.*", re.MULTILINE),
        re.compile(r"^>> Xem thêm.*", re.MULTILINE),
        re.compile(r"^Xem chi tiết tại.*", re.MULTILINE),
        re.compile(r"^>> Tham khảo.*", re.MULTILINE),
        re.compile(r"^Tham khảo.*", re.MULTILINE),
        re.compile(r"^Chọn lĩnh vực để xem văn bản liên quan.*", re.MULTILINE),
        re.compile(r"^Trân trọng.*", re.MULTILINE),
        re.compile(r"^HỎI ĐÁP PHÁP LUẬT LIÊN QUAN.*", re.MULTILINE),
        re.compile(r"^Trân trọng!*", re.MULTILINE),
        re.compile(r"^Các bài viết liên quan.*", re.MULTILINE),
        re.compile(r"^Để được tư vấn chi tiết xin vui lòng liên hệ.*", re.MULTILINE),
        re.compile(r"^TIN LIÊN QUAN.*", re.MULTILINE),
        re.compile(r"^>>.*", re.MULTILINE)
    ]
    for item in questions_and_answers1:
        answer_text = item["answer"]
        # Khởi tạo start_index với giá trị -1 (không tìm thấy)
        start_index = -1
        # Kiểm tra từng mẫu regex và cập nhật start_index nếu tìm thấy
        for pattern in patterns:
            match = pattern.search(answer_text)
            if match:
                index = match.start()
                # Nếu start_index chưa được cập nhật hoặc tìm thấy index nhỏ hơn (gần đầu chuỗi hơn)
                if start_index == -1 or index < start_index:
                    start_index = index

        # Nếu tìm thấy bất kỳ mẫu nào, xóa từ đoạn đó đến hết câu trả lời
        if start_index != -1:
            cleaned_answer = answer_text[:start_index].strip()
            item["answer"] = cleaned_answer
    # print(json.dumps(answer_text, ensure_ascii=False, indent=5))
    return json.dumps(questions_and_answers1, ensure_ascii=False, indent=5)

In [17]:
# hàm tìm tất cả câu hỏi
def find_question(soup_data):
    questions = []
    for h2_tags in find_h2_tags(soup_data):
        questions.append(h2_tags.text)
        # print(h2_tags)
    return questions

In [18]:
# hàm xử lý output
def extract_data(soup_data):
    file1_data = json.loads(processing_1(find_h2_tags(soup_data),soup_data))
    # Chỉ giữ lại các mục có câu trả lời
    filtered_questions_and_answers = [qa for qa in file1_data if 'answer' in qa and qa['answer']]
    filtered_questions_and_answers = [qa for qa in filtered_questions_and_answers if 'question' in qa and qa['question']]
    # Remove duplicates
    questions_and_answers_final = remove_duplicates_ccpls(filtered_questions_and_answers)
    return questions_and_answers_final

In [19]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = "sk-None-Y8DoDJRGH63XFz1edTyaT3BlbkFJMtiwHXQBmx5addWF1b57"

from langchain_openai import ChatOpenAI

model_gpt = ChatOpenAI(model="gpt-4")

from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [20]:
from langchain_core.messages import HumanMessage, SystemMessage

In [21]:
# Lặp qua từng dòng của DataFrame và lấy giá trị của từng dòng
# for index, row in df.iterrows():
#     input = index
input = ' Người dưới 18 tuổi có được tham gia BHXH bắt buộc không?'

postagging = postagging_keyword(input)
google_search_text = '; '.join(postagging)
# print(google_search_text)
# print(google_search_text)
gooole_request = google_search(google_search_text)
# print(gooole_request)
# gooole_request = google_search(postagging)
matching_result = find_max_result_google_search(input, gooole_request, model)
# print(matching_result)
# matching_result = find_most_similar_sentence(postagging ,gooole_request)
output = ''
# print(matching_result)
if matching_result:
    data_soup = crawl_data(matching_result)
    question_all = find_question(data_soup)
    # question = find_most_similar_question(postagging, question_all)
    question = find_max_result_question(input, question_all, model)
    # print(question)
    output = extract_data(data_soup)
result = ""
if output:
    for temp in output:
        if temp['question'] == question:
            # result = temp['answer'].strip()
            result = (temp['answer'].strip())
            # print(matching_result.get('href'))
            # if temp['ccpls']:
            #     print(temp['ccpls'])
            # break
else:
    print('')

c:\Users\My_Pc\anaconda3\envs\Minh-AI\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'luatvietnam.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [22]:
# Lặp qua từng dòng của DataFrame và lấy giá trị của từng dòng
# for index, row in df.iterrows():
#     input = index
input = 'Tách sổ đỏ cho con có mất phí không?'

postagging = postagging_keyword(input)
google_search_text = '; '.join(postagging)
# print(google_search_text)
# print(google_search_text)
gooole_request = google_search(google_search_text)
# print(gooole_request)
# gooole_request = google_search(postagging)
matching_result = find_max_result_google_search(input, gooole_request, model)
# print(matching_result)
# matching_result = find_most_similar_sentence(postagging ,gooole_request)
output = ''
# print(matching_result)
if matching_result:
    data_soup = crawl_data(matching_result)
    question_all = find_question(data_soup)
    # question = find_most_similar_question(postagging, question_all)
    question = find_max_result_question(input, question_all, model)
    # print(question)
    output = extract_data(data_soup)
result = ""
ccpl = []
if output:
    for temp in output:
        if temp['question'] == question:
            # result = temp['answer'].strip()
            result = (temp['answer'].strip())
            if temp['ccpls']:
                ccpl.append(temp['ccpls'])
            # print(matching_result.get('href'))
            # if temp['ccpls']:
            #     print(temp['ccpls'])
            # break
else:
    print('')


messages = [
    SystemMessage(content="""bạn là một luật gia sư tại việt nam, hãy trả lời trung thực, không bịa ra câu trả lời.
                  Nếu bên dưới có thông tin thì hãy dựa vào thông tin bên dưới để trả lời câu hỏi cho người dùng, sau đó hãy đưa ra kết luận cho người dùng: """ + result),
    HumanMessage(content= input),
]

result_gpt = model_gpt.invoke(messages)
print(parser.invoke(result_gpt)
if ccpl:
    for temp in ccpl:
        print(temp)
print(matching_result.get('href'))

c:\Users\My_Pc\anaconda3\envs\Minh-AI\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'luatvietnam.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


TypeError: BaseOutputParser.invoke() got an unexpected keyword argument 'stream'